In [1]:
import pandas as pd
import os
from openai import OpenAI
import time
import re

# start server first in LM Studio
# https://www.youtube.com/watch?v=2Ek0FL_Ldf4&ab_channel=echohive


In [2]:
# function to prompt model
def prompt_model(system_content, user_content):

  completion = client.chat.completions.create(
    model="local-model",
    messages=[
      {"role": "system", "content": system_content},
      {"role": "user", "content": user_content}
    ],
    temperature=0,
    max_tokens=350,
  )

  out = completion.choices[0].message.content
  return out


# function to loop over data and create output dataframe
def get_inferences(data, system_prompt, user_prompt_function):
  target_ids = data['target_id'].tolist()

  nrow = data.shape[0]

  label_list = []
  id_list = []
  for id in target_ids: 
      
      # extract index
      i = target_ids.index(id)
      print(f"Prompt {i+1} out of {nrow}")

      # get row
      r = data.iloc[i]
      post = str(r['post'])
      context = str(r['context'])
      target = str(r['target'])
    
      user_prompt = user_prompt_function(post,context,target)
      
      print(system_prompt+user_prompt)
      
      # prompt model 
      completion = prompt_model(system_prompt, user_prompt)

      label_list.append(completion)
      id_list.append(id)
      print(completion)

  df_label = pd.DataFrame()
  df_label['target_id'] = id_list
  df_label['label'] = label_list
  return df_label


In [3]:

# read data
df = pd.read_csv("../data/dt_survey_0625_with_toxicity_clean_textonly.csv")
df['post'] = df['post'].fillna("")
df['context'] = df['context'].fillna("")
df['target'] = df['target'].fillna("")

# point to local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")



# Topics 

## Drugs

In [4]:
drugs_system_prompt = '''
    In this task, you will be shown a Facebook post and two comments on this post. One comment is from person A and the other comment is from person B. 
    
    Your job is to determine if the comment by B is related to the consumption, production, or distribution of drugs. Drugs refer to substances with psychoactive effects. 

    Your answer must be a single number: 0 if irrelevant, 1 if relevant.
    
    Here are some examples with reasoning. Use the following format for your output:

    Example 1:
    Comment by B: "She is high on drugs of stupidity!"
    Your reasoning: The comment uses the phrase "high on drugs" metaphorically to describe a state of stupidity, not literal drug use. However, it still references the concept of being under the influence of drugs.
    Your answer: 1

    Example 2:
    Comment by B: "And who's fault is it that the cartels exist ?? Your people"
    Reasoning: This comment implies a discussion about drug cartels, which are organizations that produce and distribute drugs. Therefore, it is relevant to the topic of drugs.
    Your answer: 1

    Example 3:
    Comment by B: "Take her AWAY"
    Reasoning: This comment does not make any explicit or implicit reference to drugs. Hence, the comment is irrelevant to the topic of drugs.
    Your answer: 0

    '''

def drugs_user_prompt(post,context,target):
    
    user_prompt = '''Next, carefully read the following Facebook post and the comment by person A to get a sense of the context of the discussion.

    Facebook post: ''' + post + '''
    Comment by A: '''  + context + '''

    Based on this context, you must first provide reasoning for why you classify the following comment by B as either irrelevant or relevant to drugs. Reasoning: 
    
    Then, you must provide your answer as a single number: 0 if irrelevant, 1 if relevant. Your answer:
    
    Comment by B: ''' + target

    return user_prompt


drugs_df = get_inferences(df, drugs_system_prompt, drugs_user_prompt)
drugs_df.to_csv("drugs.csv")

## Suggestive content

In [ ]:
suggestive_system_prompt = '''
    In this task, you will be shown a Facebook post and two comments on this post. One comment is from person A and the other comment is from person B. 
    
    Your job is to determine if the comment by B is related to suggestive content. Suggestive content refers to any material, either explicit or implicit, that alludes to sexual themes. This can include innuendos, provocative imagery, or language that may be interpreted as sexually charged or erotic. 

    Your answer must be a single number: 0 if irrelevant, 1 if relevant.
    
    Here are some examples with reasoning. Use the following format for your output:

    Example 1:
    Comment by B: "If those prostitutes have balls."
    Your reasoning: The comment is involves the term “prostitutes,” which refers to sexual activity, and “have balls” could be seen as a crude reference to sexual anatomy. Hence, it is relevant to suggestive content.
    Your answer: 1

    Example 2:
    Comment by B: "They have hunting... And dicks on their brain!"
    Reasoning: The term "dicks" is a sexual reference. Thus, this comment involves suggestive content.
    Your answer: 1

    Example 3:
    Comment by B: "Very painful to watch."
    Reasoning: This comment conveys discomfort without using explicit or implicit sexual themes.
    Your answer: 0

    '''

def suggestive_user_prompt(post,context,target):
    
    user_prompt = '''Next, carefully read the following Facebook post and the comment by person A to get a sense of the context of the discussion.

    Facebook post: ''' + post + '''
    Comment by A: '''  + context + '''

    Based on this context, you must first provide reasoning for why you classify the following comment by B as either irrelevant or relevant to suggestive content. Reasoning: 
    
    Then, you must provide your answer as a single number: 0 if irrelevant, 1 if relevant. Your answer:
    
    Comment by B: ''' + target

    return user_prompt


suggestive_df = get_inferences(df, suggestive_system_prompt, suggestive_user_prompt)
suggestive_df.to_csv("suggestive.csv")

## Negative portrayal of Christianity

In [ ]:
christianity_system_prompt = '''
    In this task, you will be shown a Facebook post and two comments on this post. One comment is from person A and the other comment is from person B. 
    
    Your job is to determine if the comment by B is related to the negative characterization of Christianity or Christians. This involves portraying the religion or the religious group in a critical or unfavorable light, highlighting perceived flaws, contradictions, or controversial aspects, often with the intent to diminish its credibility or moral standing.
    
    Your answer must be a single number: 0 if irrelevant, 1 if relevant.
    
    Here are some examples with reasoning. Use the following format for your output:

    Example 1:
    Comment by B: "Another good 'Christian' pedophile!"
    Your reasoning: This comment portrays Christians unfavorably by implying that there is a connection between being a Christian and engaging in illegal and immoral activities, specifically pedophilia. 
    Your answer: 1

    Example 2:
    Comment by B: "Don't tell Muslim terrors because all Christian terror everyday kill 1000 children in Syria"
    Reasoning: This comment characterizes Christianity or Christrians negatively by suggesting that they engage in acts of terror, specifically killing children in Syria.
    Your answer: 1

    Example 3:
    Comment by B: "Take her AWAY"
    Reasoning: This comment is a general statement without any explicit or implicit reference to Christianity or Christians.
    Your answer: 0

    '''

def christianity_user_prompt(post,context,target):
    
    user_prompt = '''Next, carefully read the following Facebook post and the comment by person A to get a sense of the context of the discussion.

    Facebook post: ''' + post + '''
    Comment by A: '''  + context + '''

    Based on this context, you must first provide reasoning for why you classify the following comment by B as either irrelevant or relevant to the negative characterization of Christianity or Christians. Reasoning: 
    
    Then, you must provide your answer as a single number: 0 if irrelevant, 1 if relevant. Your answer:
    
    Comment by B: ''' + target

    return user_prompt


christianity_df = get_inferences(df, christianity_system_prompt, christianity_user_prompt)
christianity_df.to_csv("christianity.csv")

## Negative portrayal of America

In [ ]:
america_system_prompt = '''
    In this task, you will be shown a Facebook post and two comments on this post. One comment is from person A and the other comment is from person B. 
    
    Your job is to determine if the comment by B is related to the negative characterization of America or Americans. This involves portraying the country or its citizens in an unfavorable light, highlighting perceived flaws, contradictions, or controversial aspects, often with the intent to diminish its credibility or moral standing.
    Your answer must be a single number: 0 if irrelevant, 1 if relevant.
    
    Here are some examples with reasoning. Use the following format for your output:

    Example 1:
    Comment by B: "You mean unlike the American killers of babies, rapist, or serial killers?"
    Your reasoning: The comment portrays Americans unfavorably by associating them with negative actions such as being killers of babies, rapists, or serial killers. 
    Your answer: 1

    Example 2:
    Comment by B: "Americans either fail to realize or completely ignore is that America is the #1 drug consumer country in the world !"
    Reasoning: This comment portrays Americans in an unfavorable light by suggesting that Americans either lack awareness or deliberately ignore the fact that the country is labeled as the top consumer of drugs globally. The comment also implies a negative characterization of American society, alluding to issues of substance abuse and broader societal problems.
    Your answer: 1

    Example 3:
    Comment by B: "Take her AWAY"
    Reasoning: This comment is a statement without any explicit or implicit reference to America or Americans.
    Your answer: 0

    '''

def america_user_prompt(post,context,target):
    
    user_prompt = '''Next, carefully read the following Facebook post and the comment by person A to get a sense of the context of the discussion.

    Facebook post: ''' + post + '''
    Comment by A: '''  + context + '''

    Based on this context, you must first provide reasoning for why you classify the following comment by B as either irrelevant or relevant to the negative characterization of America or Americans. Reasoning: 
    
    Then, you must provide your answer as a single number: 0 if irrelevant, 1 if relevant. Your answer:
    
    Comment by B: ''' + target

    return user_prompt


america_df = get_inferences(df, america_system_prompt, america_user_prompt)
america_df.to_csv("america.csv")

## Anti-conservative rhetoric

In [ ]:
anticons_system_prompt = '''
    In this task, you will be shown a Facebook post and two comments on this post. One comment is from person A and the other comment is from person B. 
    
    Your job is to determine if the comment by B is related to anti-conservative rhetoric. This involves drawing on themes that portray conservatism or conservatives in an unfavorable light, highlighting perceived flaws, contradictions, or controversial aspects, often with the intent to diminish its credibility or moral standing. This includes, but is not limited to, accusations of being out of touch with the needs and concerns of certain demographic groups and minorities, association with extremist political views, accusations of racism, or the labeling as heartless or uncompassionate.
    Your answer must be a single number: 0 if irrelevant, 1 if relevant.
    
    Here are some examples with reasoning. Use the following format for your output:

    Example 1:
    Comment by B: "Conservatives can also be whiny. "
    Your reasoning: The comment portrays conservatives as complaining, overly critical, or expressing dissatisfaction in a way that may be perceived as annoying or unproductive.
    Your answer: 1

    Example 2:
    Comment by B: "Dumb as hillbilly racist. She should behead you deplorable. k.k.k."
    Reasoning: This comment employs derogatory language, combines stereotypes (such as "hillbilly") with an accusation of racism.
    Your answer: 1

    Example 3:
    Comment by B: "Take her AWAY"
    Reasoning: This comment is a general statement without any explicit or implicit reference to America or Americans.
    Your answer: 0

    '''

def anticons_user_prompt(post,context,target):
    
    user_prompt = '''Next, carefully read the following Facebook post and the comment by person A to get a sense of the context of the discussion.

    Facebook post: ''' + post + '''
    Comment by A: '''  + context + '''

    Based on this context, you must first provide reasoning for why you classify the following comment by B as either irrelevant or relevant to anti-conservative rhetoric. Reasoning: 
    
    Then, you must provide your answer as a single number: 0 if irrelevant, 1 if relevant. Your answer:
    
    Comment by B: ''' + target

    return user_prompt


anticons_df = get_inferences(df, anticons_system_prompt, anticons_user_prompt)
anticons_df.to_csv("anticonservative.csv")